In [1]:
import pandas as pd
import numpy as np
import sys
from sklearn.metrics.pairwise import pairwise_distances

In [2]:
#temp name change to short from short_names
short=['bone','chemo','rad','cpr','hrtman','bpcho','diab_care',
                  'diab_edu','dialysis','infus','gene','pict','lab','xray','glass_a','glass_c',
                  'eye_a','eye_c','dme','g_tube','hab','home','hospice','pain','osteo',
                  'prosth','skill_rn','er','ambu','hosp','inpt','all_inj','all_test','breath',
                  'prevent','transplant','l_d','nutr','preg_m','prenat','well_b','depr','ment_off',
                  'ment_in','ment_out','not_doc','out_surg','out_ambu','pcp','foot','spec','tele','urgent',
                  'back','observ','rehab_out','reconst','pt_rehab','speech','drug_in','drug_out',
                  'drug_off','chiro','fit','gym','o2','nutr_counc','tmj','kg_m']
# 'PlanId'
new_cols = ['Bone Marrow Transplant','Chemotherapy','Radiation',
                 'Cardiac and Pulmonary Rehabilitation','Heart Disease Management Program',
                 'High Blood Pressure & High Cholesterol Management Program','Diabetes Care Management',
                 'Diabetes Education','Dialysis','Infusion Therapy','Genetic Testing Lab Services',
                 'Imaging (CT/PET Scans, MRIs)','Laboratory Outpatient and Professional Services',
                 'X-rays and Diagnostic Imaging','Eye Glasses for Adults','Eye Glasses for Children',
                 'Routine Eye Exam (Adult)','Routine Eye Exam for Children',
                 'Durable Medical Equipment','Enteral/Parenteral and Oral Nutrition Therapy',
                 'Habilitation Services','Home Health Care Services','Hospice Services',
                 'Pain Management Program','Osteoporosis','Prosthetic Devices',
                 'Skilled Nursing Facility','Emergency Room Services',
                 'Emergency Transportation/Ambulance',
                 'Inpatient Hospital Services (e.g., Hospital Stay)',
                 'Inpatient Physician and Surgical Services','Allergy Injections',
                 'Allergy Testing','Asthma Management Program',
                 'Preventive Care/Screening/Immunization','Transplant',
                 'Delivery and All Inpatient Services for Maternity Care','Nutrition/Formulas',
                 'Pregnancy Management Program','Prenatal and Postnatal Care',
                 'Well Baby Visits and Care','Depression Management Program',
                 'Mental Health Office Visit','Mental/Behavioral Health Inpatient Services',
                 'Mental/Behavioral Health Outpatient Services',
                 'Other Practitioner Office Visit (Nurse, Physician Assistant)',
                 'Outpatient Facility Fee (e.g., Ambulatory Surgery Center)',
                 'Outpatient Surgery Physician/Surgical Services',
                 'Primary Care Visit to Treat an Injury or Illness','Routine Foot Care',
                 'Specialist Visit','Telehealth','Urgent Care Centers or Facilities',
                 'Low Back Pain Management Program','Outpatient Observation',
                 'Outpatient Rehabilitation Services','Reconstructive Surgery',
                 'Rehabilitative Occupational and Rehabilitative Physical Therapy',
                 'Rehabilitative Speech Therapy','Substance Abuse Disorder Inpatient Services',
                 'Substance Abuse Disorder Outpatient Services','Substance Abuse Office Visit',
                 'Chiropractic Care','Fitness Center Membership','Gym Access',
                 'Hyperbaric Oxygen Therapy','Nutritional Counseling',
                 'Treatment for Temporomandibular Joint Disorders',
                 'Weight Loss Management Program']

rename_d = dict(zip(new_cols,short))

def make_plan_matrix():
    '''
        imports needed the plan detail files, merges them, reorders to match 
        the flask app, then outputs two versions. 

            plans_matrix_df is kept with the index and column names so it 
            can be referred back to provide the plan names.

            plans_matrix is transformed to a matrix for actual calculations.

    '''

    attr = pd.read_csv('../data/clean_files/attr_FL.csv')
    ben = pd.read_csv('../data/clean_files/benefits_covered_FL.csv')

    #attr.drop(['Unnamed: 0','Issuer_ID'], axis=1, inplace=True)
    #attr.drop(['Issuer_Denial_rate','Plan_denial_rate','Disenrollment_Rate'], axis=1, inplace=True)


    #attr=attr.sort_values(by=['IssuerId', 'ServiceAreaId'])
    merged_ben_attr = ben.merge(attr, how='left', on='PlanId')
    merge_clean= merged_ben_attr.dropna()
    merge_clean['Osteo']= merge_clean['Osteoporosis']+merge_clean['Osteoporosis Treatment']#     merge_clean3=merge_clean.drop(['Osteoporosis','Osteoporosis Treatment','StandardComponentId','IssuerId','NetworkId',
#                                    'ServiceAreaId','PlanType','MetalLevel','PlanVariantMarketingName','Issuer_Name',
#                                    'No Management Program'],axis=1)
    merge_clean3=merge_clean.drop(['Osteoporosis','Osteoporosis Treatment','PlanType','MetalLevel',
                                  'PlanVariantMarketingName'],axis=1)

    merge_clean3.rename(columns={'Osteo':'Osteoporosis'}, inplace=True)
    '''
        this will replace features - exclusions are misbehaving so they have
        been commented out of flask

        new_cols_all = ['PlanId','Bone Marrow Transplant','Chemotherapy','Radiation',
        'Cardiac and Pulmonary Rehabilitation','Heart Disease Management Program',
        'High Blood Pressure & High Cholesterol Management Program','Diabetes Care Management',
        'Diabetes Education','Dialysis','Infusion Therapy','Genetic Testing Lab Services',
        'Imaging (CT/PET Scans, MRIs)','Laboratory Outpatient and Professional Services',
        'X-rays and Diagnostic Imaging','Eye Glasses for Adults','Eye Glasses for Children',
        'Routine Eye Exam (Adult)','Routine Eye Exam for Children',
        'Durable Medical Equipment','Enteral/Parenteral and Oral Nutrition Therapy',
        'Habilitation Services','THIS IS WHERE HEARING AIDS GOES',
        'Home Health Care Services','Hospice Services',
        'THIS IS WHERE LONG TERM NURSING GOES','Pain Management Program','THIS IS WHERE PRIVATE NURSE GOES',
        'Osteoporosis','Prosthetic Devices','Skilled Nursing Facility',
        'Emergency Room Services','Emergency Transportation/Ambulance',
        'Inpatient Hospital Services (e.g., Hospital Stay)','Inpatient Physician and Surgical Services',
        'Allergy Injections','Allergy Testing','Asthma Management Program',
        'Preventive Care/Screening/Immunization','Transplant','ABORTION GOES HERE',
        'Delivery and All Inpatient Services for Maternity Care','INFERTILITY GOES HERE',
        'NEWBORN HEARING GOES HERE', 'NEWBORN OTHER GOES HERE','Nutrition/Formulas',
        'Pregnancy Management Program','Prenatal and Postnatal Care',
        'Well Baby Visits and Care','Depression Management Program',
        'Mental Health Office Visit','Mental/Behavioral Health Inpatient Services',
        'Mental/Behavioral Health Outpatient Services', 'MENTAL HEALTH OTHER GOES HERE',
        'Other Practitioner Office Visit (Nurse, Physician Assistant)',
        'Outpatient Facility Fee (e.g., Ambulatory Surgery Center)',
        'Outpatient Surgery Physician/Surgical Services',
        'Primary Care Visit to Treat an Injury or Illness','Routine Foot Care',
        'Specialist Visit','Telehealth','Urgent Care Centers or Facilities',
        'Low Back Pain Management Program','Outpatient Observation',
        'Outpatient Rehabilitation Services','Reconstructive Surgery',
        'Rehabilitative Occupational and Rehabilitative Physical Therapy',
        'Rehabilitative Speech Therapy','Substance Abuse Disorder Inpatient Services',
        'Substance Abuse Disorder Outpatient Services','Substance Abuse Office Visit',
        'ACUPUNCTURE GOES HERE','BARIATRIC GOES HERE','Chiropractic Care','COSMETIC GOES HERE',
        'Fitness Center Membership','Gym Access','Hyperbaric Oxygen Therapy','Nutritional Counseling',
        'Treatment for Temporomandibular Joint Disorders','Weight Loss Management Program']
    '''
    features= ['PlanId','Bone Marrow Transplant','Chemotherapy','Radiation',
               'Cardiac and Pulmonary Rehabilitation','Heart Disease Management Program',
               'High Blood Pressure & High Cholesterol Management Program','Diabetes Care Management',
               'Diabetes Education','Dialysis','Infusion Therapy','Genetic Testing Lab Services',
               'Imaging (CT/PET Scans, MRIs)','Laboratory Outpatient and Professional Services',
               'X-rays and Diagnostic Imaging','Eye Glasses for Adults','Eye Glasses for Children',
               'Routine Eye Exam (Adult)','Routine Eye Exam for Children',
               'Durable Medical Equipment','Enteral/Parenteral and Oral Nutrition Therapy',
               'Habilitation Services','Home Health Care Services','Hospice Services',
               'Pain Management Program','Osteoporosis','Prosthetic Devices',
               'Skilled Nursing Facility','Emergency Room Services',
               'Emergency Transportation/Ambulance',
               'Inpatient Hospital Services (e.g., Hospital Stay)',
               'Inpatient Physician and Surgical Services','Allergy Injections',
               'Allergy Testing','Asthma Management Program',
               'Preventive Care/Screening/Immunization','Transplant',
               'Delivery and All Inpatient Services for Maternity Care','Nutrition/Formulas',
               'Pregnancy Management Program','Prenatal and Postnatal Care',
               'Well Baby Visits and Care','Depression Management Program',
               'Mental Health Office Visit','Mental/Behavioral Health Inpatient Services',
               'Mental/Behavioral Health Outpatient Services',
               'Other Practitioner Office Visit (Nurse, Physician Assistant)',
               'Outpatient Facility Fee (e.g., Ambulatory Surgery Center)',
               'Outpatient Surgery Physician/Surgical Services',
               'Primary Care Visit to Treat an Injury or Illness','Routine Foot Care',
               'Specialist Visit','Telehealth','Urgent Care Centers or Facilities',
               'Low Back Pain Management Program','Outpatient Observation',
               'Outpatient Rehabilitation Services','Reconstructive Surgery',
               'Rehabilitative Occupational and Rehabilitative Physical Therapy',
               'Rehabilitative Speech Therapy','Substance Abuse Disorder Inpatient Services',
               'Substance Abuse Disorder Outpatient Services','Substance Abuse Office Visit',
               'Chiropractic Care','Fitness Center Membership','Gym Access',
               'Hyperbaric Oxygen Therapy','Nutritional Counseling',
               'Treatment for Temporomandibular Joint Disorders',
               'Weight Loss Management Program']
    plans_matrix_df= merge_clean3[features]
    plans_matrix_df["IssuerId"]=plans_matrix_df['PlanId'].str.slice(stop=5)
    plans_matrix_df.drop_duplicates(['IssuerId'], inplace=True)
    #plans_matrix_df.set_index('PlanId')
    plans_matrix_df.drop(['PlanId'],axis=1, inplace=True)
    plans_matrix_df=plans_matrix_df.astype(int)
    plans_matrix_df.to_pickle('../models/florida_jaccard.pkl')
   
    plans=plans_matrix_df['IssuerId']
    plans_matrix_df.drop(['IssuerId'],axis=1, inplace=True)

   
    return (plans_matrix_df,plans)
            
def new_customer(short_names):
    customer_dict={}
    #print(len(short_names),'short',short_names)
    for short_name in short_names:
        customer_dict[short_name]=-1
   # print('cust_dict',customer_dict.keys())
    return customer_dict


def make_customer_row(customer_dict,customer_choices):
    for benefit in customer_choices:
        if benefit in customer_dict.keys():
             customer_dict[benefit]=1
                
    #print(len(customer_dict.keys()),'cust_dict',customer_dict.keys())        
    return customer_dict


def benefit_dict(short_names,features):
    benefit_dict= dict(zip(short_names, features))
    #print (len(benefit_dict.keys()),'ben__dict',benefit_dict.keys())
    return benefit_dict

def jaccard(plans_matrix,customer_dict):
    a=list(plans_matrix.columns)
    
    z=list(customer_dict.keys())
    li_dif = [i for i in a + z if i not in z or i not in a] 
#     print(li_dif)
#     print(plans_matrix.columns)
#     print(len(a), len(z))
#     print(customer_dict.keys())
    
    jaccard_matrix = plans_matrix.append(customer_dict, ignore_index=True)
    jaccard_array= jaccard_matrix.to_numpy()
    #print(jaccard_array)
    #print(jaccard_matrix.head())
    jaccard_calc = 1 - pairwise_distances(jaccard_array, metric = "hamming")
    #jaccard_similarity = pd.DataFrame(jaccard_calc, index=plans_matrix.index, columns=plans_matrix.index)
    jaccard_similarity = pd.DataFrame(jaccard_calc)
#     print(jaccard_similarity.info())

    match = jaccard_similarity.iloc[[-1]]
   
    return jaccard_similarity


def narrow_plans():
    zip=zip

    pass

def analysis(listx,renamed):
    
    customer_choices=listx
    matrix,plans = make_plan_matrix()
    matrix = matrix.rename(columns=renamed)
    b=new_customer(short)
    c=make_customer_row(b,customer_choices)
    d=benefit_dict(short,new_cols)
    
    e=jaccard(matrix,c)
    return e,plans
#     print ('Howdy')
#     return (1+2)







    
    

In [21]:
example=['infus','gene','pict','lab','xray','glass_a','glass_c','eye_a','eye_c','dme','g_tube','hab','home','hosp','pain','osteo','osteo',
         'prosth','skill_rn','er','ambu','hosp','inpt','all_inj','all_test','breath','prevent','transplant',
         'l_d','nutr']
#print(len(example))
df,names=analysis(example,rename_d)

IssuerIDs=dict({0:12379,1:16842,2:21663, 3:30252, 4:36194,5:40572,6:48121,7:54172, 8:56503, 9:'User'})
df['Plan']=[12379,16842,21663,30252,36194,40572,48121,54172,56503,'User']
print(df)
df=df.rename(columns=IssuerIDs)
print(df)
df=df.sort_values('User',ascending = False)
df=df.iloc[1:]
best_to_worst=df[['Plan','User']]


best_to_worst
# #df=df.drop(9, axis=1)
# match = df.iloc[[-1]]
# print(df,names)

# df.columns=names
# df
# #df.info()



          0         1         2         3         4         5         6  \
0  1.000000  0.869565  0.956522  0.869565  0.811594  0.956522  0.971014   
1  0.869565  1.000000  0.855072  1.000000  0.768116  0.826087  0.898551   
2  0.956522  0.855072  1.000000  0.855072  0.768116  0.942029  0.956522   
3  0.869565  1.000000  0.855072  1.000000  0.768116  0.826087  0.898551   
4  0.811594  0.768116  0.768116  0.768116  1.000000  0.768116  0.782609   
5  0.956522  0.826087  0.942029  0.826087  0.768116  1.000000  0.927536   
6  0.971014  0.898551  0.956522  0.898551  0.782609  0.927536  1.000000   
7  0.898551  0.971014  0.884058  0.971014  0.797101  0.855072  0.898551   
8  0.855072  0.927536  0.869565  0.927536  0.724638  0.811594  0.884058   
9  0.289855  0.333333  0.275362  0.333333  0.347826  0.260870  0.289855   

          7         8         9   Plan  
0  0.898551  0.855072  0.289855  12379  
1  0.971014  0.927536  0.333333  16842  
2  0.884058  0.869565  0.275362  21663  
3  0.97101

,Plan,User
4,36194,0.347826
1,16842,0.333333
3,30252,0.333333
7,54172,0.318841
8,56503,0.318841
0,12379,0.289855
6,48121,0.289855
2,21663,0.275362
5,40572,0.260870


In [4]:
sum(len(df)-df.count())

0

In [5]:
# sort eigenvalue in decreasing order
idx = np.argsort(match.values)[0]

plans = names[idx]

    # sort eigenvectors according to same indeplqnw

print(match[idx])
#print(plans)

np.unique(match.values,return_counts=True)

NameError: name 'match' is not defined

attr = pd.read_csv('../data/clean_files/attr_FL.csv')
ben = pd.read_csv('../data/clean_files/benefits_covered_FL.csv')

#attr.drop(['Unnamed: 0','Issuer_ID'], axis=1, inplace=True)
#attr.drop(['Issuer_Denial_rate','Plan_denial_rate','Disenrollment_Rate'], axis=1, inplace=True)


attr=attr.sort_values(by=[ServiceAreaId'])
merged_ben_attr = ben.merge(attr, how='left', on='PlanId')
merge_clean= merged_ben_attr.dropna()
merge_clean['Osteo']= merge_clean['Osteoporosis']+merge_clean['Osteoporosis Treatment']

merge_clean3=merge_clean.drop(['Osteoporosis','Osteoporosis Treatment','StandardComponentId','IssuerId','NetworkId',
                                   'ServiceAreaId','PlanType','MetalLevel','PlanVariantMarketingName','Issuer_Name',
                                   'No Management Program'],axis=1)

merge_clean3.rename(columns={'Osteo':'Osteoporosis'}, inplace=True)

features= ['PlanId','Bone Marrow Transplant','Chemotherapy','Radiation',
                 'Cardiac and Pulmonary Rehabilitation','Heart Disease Management Program',
                 'High Blood Pressure & High Cholesterol Management Program','Diabetes Care Management',
                 'Diabetes Education','Dialysis','Infusion Therapy','Genetic Testing Lab Services',
                 'Imaging (CT/PET Scans, MRIs)','Laboratory Outpatient and Professional Services',
                 'X-rays and Diagnostic Imaging','Eye Glasses for Adults','Eye Glasses for Children',
                 'Routine Eye Exam (Adult)','Routine Eye Exam for Children',
                 'Durable Medical Equipment','Enteral/Parenteral and Oral Nutrition Therapy',
                 'Habilitation Services','Home Health Care Services','Hospice Services',
                 'Pain Management Program','Osteoporosis','Prosthetic Devices',
                 'Skilled Nursing Facility','Emergency Room Services',
                 'Emergency Transportation/Ambulance',
                 'Inpatient Hospital Services (e.g., Hospital Stay)',
                 'Inpatient Physician and Surgical Services','Allergy Injections',
                 'Allergy Testing','Asthma Management Program',
                 'Preventive Care/Screening/Immunization','Transplant',
                 'Delivery and All Inpatient Services for Maternity Care','Nutrition/Formulas',
                 'Pregnancy Management Program','Prenatal and Postnatal Care',
                 'Well Baby Visits and Care','Depression Management Program',
                 'Mental Health Office Visit','Mental/Behavioral Health Inpatient Services',
                 'Mental/Behavioral Health Outpatient Services',
                 'Other Practitioner Office Visit (Nurse, Physician Assistant)',
                 'Outpatient Facility Fee (e.g., Ambulatory Surgery Center)',
                 'Outpatient Surgery Physician/Surgical Services',
                 'Primary Care Visit to Treat an Injury or Illness','Routine Foot Care',
                 'Specialist Visit','Telehealth','Urgent Care Centers or Facilities',
                 'Low Back Pain Management Program','Outpatient Observation',
                 'Outpatient Rehabilitation Services','Reconstructive Surgery',
                 'Rehabilitative Occupational and Rehabilitative Physical Therapy',
                 'Rehabilitative Speech Therapy','Substance Abuse Disorder Inpatient Services',
                 'Substance Abuse Disorder Outpatient Services','Substance Abuse Office Visit',
                 'Chiropractic Care','Fitness Center Membership','Gym Access',
                 'Hyperbaric Oxygen Therapy','Nutritional Counseling',
                 'Treatment for Temporomandibular Joint Disorders',
                 'Weight Loss Management Program']
plans_matrix_df= merge_clean3[features]

plans_matrix_df.set_index('PlanId')
plans_matrix_df.drop('PlanId',axis=1, inplace=True)
plans_matrix_df=plans_matrix_df.astype(int)
print(len(features))

In [ ]:
a=['bone', 'chemo', 'rad', 'cpr', 'hrtman', 'bpcho', 'diab_care', 'diab_edu', 'dialysis', 'infus', 'gene', 'pict', 'lab', 'xray', 'glass_a', 'glass_c', 'eye_a', 'eye_c', 'dme', 'g_tube', 'hab', 'home', 'hospice', 'pain', 'osteo', 'prosth', 'skill_rn', 'er', 'ambu', 'hosp', 'inpt', 'all_inj', 'all_test', 'breath', 'prevent', 'transplant', 'l_d', 'nutr', 'preg_m', 'prenat', 'well_b', 'depr', 'ment_off', 'ment_in', 'ment_out', 'not_doc', 'out_surg', 'pcp', 'foot', 'spec', 'tele', 'urgent', 'back', 'observ', 'rehab_out', 'reconst', 'pt_rehab', 'speech', 'drug_in', 'drug_out', 'drug_off', 'chiro', 'fit', 'gym', 'o2', 'nutr_counc', 'tmj', 'kg_m']
z=['bone', 'chemo', 'rad', 'cpr', 'hrtman', 'bpcho', 'diab_care', 'diab_edu', 'dialysis', 'infus', 'gene', 'pict', 'lab', 'xray', 'glass_a', 'glass_c', 'eye_a', 'eye_c', 'dme', 'g_tube', 'hab', 'home', 'hospice', 'pain', 'osteo', 'prosth', 'skill_rn', 'er', 'ambu', 'hosp', 'inpt', 'all_inj', 'all_test', 'breath', 'prevent', 'transplant', 'l_d', 'nutr', 'preg_m', 'prenat', 'well_b', 'depr', 'ment_off', 'ment_in', 'ment_out', 'not_doc', 'out_surg', 'pcp', 'foot', 'spec', 'tele', 'urgent', 'back', 'observ', 'rehab_out', 'reconst', 'pt_rehab', 'speech', 'drug_in', 'drug_out', 'drug_off', 'chiro', 'fit', 'gym', 'o2', 'nutr_counc', 'tmj', 'kg_m']
li_dif = [i for i in a + z if i not in z or i not in a] 

In [ ]:

attr = pd.read_csv('../data/clean_files/attr_FL.csv')
ben = pd.read_csv('../data/clean_files/benefits_covered_FL.csv')
attr.info()
#attr.drop(['Unnamed: 0','Issuer_ID'], axis=1, inplace=True)

In [ ]:
zip=33101
area_import = pd.read_csv('../data/clean_files/area_FL.csv')
area=area_import.copy()
area.drop(['Unnamed: 0','County','County_Name','ServiceAreaId'], axis=1, inplace=True)
area.dropna(how='any', inplace=True)
area['IssuerId']=area['IssuerId'].astype(int)
Issuers=area[area['IssuerId']==zip]


In [10]:

matrix=pd.read_pickle('../models/florida_jaccard.pkl')
plans=matrix['IssuerId']
matrix.drop(['IssuerId'],axis=1, inplace=True)
matrix

,Bone Marrow Transplant,Chemotherapy,Radiation,Cardiac and Pulmonary Rehabilitation,Heart Disease Management Program,High Blood Pressure & High Cholesterol Management Program,Diabetes Care Management,Diabetes Education,Dialysis,Infusion Therapy,...,Substance Abuse Disorder Inpatient Services,Substance Abuse Disorder Outpatient Services,Substance Abuse Office Visit,Chiropractic Care,Fitness Center Membership,Gym Access,Hyperbaric Oxygen Therapy,Nutritional Counseling,Treatment for Temporomandibular Joint Disorders,Weight Loss Management Program
0,0,1,1,0,1,0,0,1,1,1,...,1,1,0,1,0,0,0,1,1,0
152,1,1,1,0,1,1,1,1,1,1,...,1,1,0,1,0,0,0,1,1,0
272,0,1,1,0,1,1,0,1,1,0,...,1,1,0,1,0,0,0,1,1,0
587,1,1,1,0,1,1,1,1,1,1,...,1,1,0,1,0,0,0,1,1,0
725,0,1,1,1,1,0,0,1,1,1,...,1,1,1,1,1,0,1,1,1,0
951,0,1,1,0,0,0,0,1,1,0,...,1,1,0,1,0,0,0,1,1,0
1001,0,1,1,0,1,1,0,1,1,1,...,1,1,0,1,0,0,0,1,1,0
1089,1,1,1,0,1,1,1,1,1,1,...,1,1,0,1,0,0,0,1,1,0
1130,1,1,1,0,1,1,1,1,1,1,...,1,1,0,1,0,1,0,1,1,1


In [12]:
area_import = pd.read_csv('../data/clean_files/area_FL.csv')
area=area_import.copy()
area.drop(['Unnamed: 0','County','County_Name','ServiceAreaId'], axis=1, inplace=True)
area.dropna(how='any', inplace=True)
area['IssuerId']=area['IssuerId'].astype(int)
area=area[area['zipcode'] ==33101]
    

In [14]:
area.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6 entries, 4182 to 4187
Data columns (total 2 columns):
zipcode     6 non-null int64
IssuerId    6 non-null int64
dtypes: int64(2)
memory usage: 144.0 bytes


In [19]:
matrix=pd.read_pickle('../models/florida_jaccard.pkl')


area_plans=area.merge(matrix, how='left', on='IssuerId')
area_plans.drop('zipcode', axis=1, inplace=True)
area_plans



#plans=matrix['IssuerId']



#matrix.drop(['IssuerId'],axis=1, inplace=True)
#matrix

,IssuerId,Bone Marrow Transplant,Chemotherapy,Radiation,Cardiac and Pulmonary Rehabilitation,Heart Disease Management Program,High Blood Pressure & High Cholesterol Management Program,Diabetes Care Management,Diabetes Education,Dialysis,...,Substance Abuse Disorder Inpatient Services,Substance Abuse Disorder Outpatient Services,Substance Abuse Office Visit,Chiropractic Care,Fitness Center Membership,Gym Access,Hyperbaric Oxygen Therapy,Nutritional Counseling,Treatment for Temporomandibular Joint Disorders,Weight Loss Management Program
0,16842,1,1,1,0,1,1,1,1,1,...,1,1,0,1,0,0,0,1,1,0
1,21663,0,1,1,0,1,1,0,1,1,...,1,1,0,1,0,0,0,1,1,0
2,30252,1,1,1,0,1,1,1,1,1,...,1,1,0,1,0,0,0,1,1,0
3,40572,0,1,1,0,0,0,0,1,1,...,1,1,0,1,0,0,0,1,1,0
4,54172,1,1,1,0,1,1,1,1,1,...,1,1,0,1,0,0,0,1,1,0
5,16842,1,1,1,0,1,1,1,1,1,...,1,1,0,1,0,0,0,1,1,0
